In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
#os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_dual as aformer
#import src.aformer_TF as aformer
from src.layers.layers_dual import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_aformer_TF_expression as training_utils

from scipy import stats


2022-10-11 15:55:43.210753: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-11 15:55:43.210800: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-1')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy=\
        tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    options.experimental_threading.max_intra_op_parallelism=1
    mixed_precision.set_global_policy('mixed_bfloat16')
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 16 # batch size 24, use LR ~ 2.5 e -04
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS
    

2022-10-11 15:55:45.300415: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-11 15:55:45.300483: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-11 15:55:45.300505: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2022-10-11 15:55:45.638427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
2022-10-11 15:55:45.648338: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.30.154.90:8470}
2022-10-11 15:55:45.648382: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:40722}
2022-10-11 15:55:45.663853: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initialize GrpcChannelCache for job worker -> {0 -> 10.30.154.90:8470}
2022-10-11 15:55:45.663898: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:272] Initializ

INFO:tensorflow:Initializing the TPU system: node-1


INFO:tensorflow:Initializing the TPU system: node-1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
with strategy.scope():
    data_dict_tr,data_dict_val = training_utils.return_distributed_iterators("gs://picard-testing-176520/seqtoatac_98k_73kstride_blacklist0.25/preprocessed",
                                                                             "gs://picard-testing-176520/seqtoatac_98k_73kstride_blacklist0.25/preprocessed",
                                                                             "atac_only",
                                                                              GLOBAL_BATCH_SIZE,
                                                                              98304,
                                                                              768,
                                                                              128,
                                                                             20,
                                                                               8,
                                                                               1,
                                                                               strategy,
                                                                               options)


In [4]:
with strategy.scope():
    inits = training_utils.get_initializers("/home/jupyter/dev/BE_CD69_paper_2022/enformer_fine_tuning/checkpoint/sonnet_weights")
    model = aformer.aformer(kernel_transformation = 'softmax_kernel_transformation',
                            dropout_rate = 0.50,
                            attention_dropout_rate = 0.25,
                            tf_dropout_rate=0.50,
                            pointwise_dropout_rate=0.15,
                            input_length = 98304,
                            atac_length_uncropped = 768,
                            atac_output_length = 448,
                             num_heads = 8,
                             numerical_stabilizer = 0.001,
                             nb_random_features = 256,
                             hidden_size = 1600,
                             transformer_depth_1 = 3,
                             transformer_depth_2 = 3,
                             shared_transformer_depth = 6,
                             pre_transf_channels = 1600,
                             d_model = 1600,
                             TF_inputs=256,
                             norm=True,
                             dim = 200, 
                             max_seq_length = 768,
                             use_rot_emb = True,
                             use_mask_pos = False, 
                             normalize = True,
                             seed = 3,
                             load_init=True,
                             inits=inits,
                             #filter_list=[256,294,338,388,446,512],
                             freeze_conv_layers=True)
    

In [5]:
with strategy.scope():
    scheduler1= tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=1.0e-07,
        decay_steps=1000, alpha=1.0)
    scheduler1=WarmUp(initial_learning_rate=5.0e-07,
                                 warmup_steps=50,
                                 decay_schedule_fn=scheduler1)
    optimizer1 = tfa.optimizers.AdamW(learning_rate=scheduler1,
                                           weight_decay=1.0e-08)
    
    scheduler2= tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=7.5e-05,
    decay_steps=1000, alpha=1.0)
    scheduler2=WarmUp(initial_learning_rate=7.5e-05,
                                 warmup_steps=50,
                                 decay_schedule_fn=scheduler2)
    optimizer2 = tfa.optimizers.AdamW(learning_rate=scheduler2,
                                          weight_decay=7.5e-06)

    optimizers_in=optimizer1,optimizer2
    

In [6]:
with strategy.scope():
    metric_dict = {}
    dist_train_step_atac, dist_val_step_atac,\
            dist_train_step_rna,dist_val_step_rna,\
                dist_train_step_both,dist_val_step_both, build_step, metric_dict = training_utils.return_train_val_functions(model,
                                                                                               optimizers_in,
                                                                                               strategy,
                                                                                               metric_dict,
                                                                                               300, 
                                                                                               150,
                                                                                               50,
                                                                                               GLOBAL_BATCH_SIZE,
                                                                                               5.0,
                                                                                               448,
                                                                                               120,
                                                                                               BATCH_SIZE_PER_REPLICA,
                                                                                               rna_loss_scale=None)


In [ ]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = [] 
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 15):
        if epoch_i == 1:
            build_step(data_dict_val)
            total_params = 0
            for k in model.trainable_variables:
                var = k.values[0]
                print(var.name)
                print(tf.size(var))
                total_params += tf.size(var)
                
            print('total params: ' + str(total_params)) 
        start = time.time()

        
        #if epoch_i > 2 : 
        dist_train_step_atac(data_dict_tr)

        print('hg_train_loss: ' + str(metric_dict['hg_tr'].result().numpy()))

        dist_val_step_atac(data_dict_val)

        print('hg_val_loss: ' + str(metric_dict['hg_val'].result().numpy()))
        print('hg_ATAC_pearsons: ' + str(metric_dict['hg_pearsonsR_ATAC'].result()['PearsonR'].numpy()))
        print('hg_ATAC_R2: ' + str(metric_dict['hg_R2_ATAC'].result()['R2'].numpy()))
        
        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))

        for key, item in metric_dict.items():
            item.reset_state()

aformer/tf_module/dense/kernel:0
tf.Tensor(419072, shape=(), dtype=int32)
aformer/tf_module/dense_1/kernel:0
tf.Tensor(32768, shape=(), dtype=int32)
aformer/tf_module/dense_2/kernel:0
tf.Tensor(8192, shape=(), dtype=int32)
aformer/conv_mix_block/sync_batch_normalization_13/gamma:0
tf.Tensor(1600, shape=(), dtype=int32)
aformer/conv_mix_block/sync_batch_normalization_13/beta:0
tf.Tensor(1600, shape=(), dtype=int32)
aformer/conv_mix_block/conv1d_14/kernel:0
tf.Tensor(2560000, shape=(), dtype=int32)
aformer/conv_mix_block/conv1d_14/bias:0
tf.Tensor(1600, shape=(), dtype=int32)
aformer/dim_reduce_block2/sync_batch_normalization_14/gamma:0
tf.Tensor(1603, shape=(), dtype=int32)
aformer/dim_reduce_block2/sync_batch_normalization_14/beta:0
tf.Tensor(1603, shape=(), dtype=int32)
aformer/dim_reduce_block2/conv1d_15/kernel:0
tf.Tensor(2564800, shape=(), dtype=int32)
aformer/dim_reduce_block2/conv1d_15/bias:0
tf.Tensor(1600, shape=(), dtype=int32)
aformer/shared_transformer/transformer_layer/laye

In [ ]:
17182640 + 65602151 + 8584289